In [2]:
!pip install dlib opencv-python


  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlib: filename=dlib-20.0.0-cp312-cp312-linux_x86_64.whl size=4158259 sha256=40f7d4373f34febedb1ce003ed73f1a3c6578198f695e78ee8385a19ccd4babf
  Stored in directory: /home/himel/.cache/pip/wheels/35/bc/f4/3551aa7a295bf59b6cbf9cb588197b668052e5ec92a02aff7f
Successfully built dlib


In [1]:
import cv2
import dlib
import math

# Load the pre-trained facial landmark predictor model
predictor_path = '/home/himel/Downloads/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Eye aspect ratio calculation
def eye_aspect_ratio(eye):
    # Calculate the Euclidean distances between the two sets of vertical eye landmarks
    A = math.sqrt((eye[1].x - eye[5].x)**2 + (eye[1].y - eye[5].y)**2)
    B = math.sqrt((eye[2].x - eye[4].x)**2 + (eye[2].y - eye[4].y)**2)
    
    # Calculate the Euclidean distance between the horizontal eye landmarks
    C = math.sqrt((eye[0].x - eye[3].x)**2 + (eye[0].y - eye[3].y)**2)
    
    # Return the Eye Aspect Ratio (EAR)
    ear = (A + B) / (2.0 * C)
    return ear

# Mouth aspect ratio calculation
def mouth_aspect_ratio(mouth):
    # Calculate the distances between key landmarks on the mouth
    A = math.sqrt((mouth[2].x - mouth[10].x)**2 + (mouth[2].y - mouth[10].y)**2)
    B = math.sqrt((mouth[4].x - mouth[8].x)**2 + (mouth[4].y - mouth[8].y)**2)
    
    # Calculate the horizontal distance between the mouth
    C = math.sqrt((mouth[0].x - mouth[6].x)**2 + (mouth[0].y - mouth[6].y)**2)
    
    # Return the Mouth Aspect Ratio (MAR)
    mar = (A + B) / (2.0 * C)
    return mar

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    # Loop through each detected face
    for face in faces:
        # Get the landmarks for the face
        landmarks = predictor(gray, face)

        # Get the coordinates for the eye landmarks (left and right eyes)
        left_eye = [landmarks.part(i) for i in range(36, 42)]  # Left eye landmarks
        right_eye = [landmarks.part(i) for i in range(42, 48)]  # Right eye landmarks
        
        # Get the coordinates for the mouth landmarks
        mouth = [landmarks.part(i) for i in range(48, 68)]  # Mouth landmarks

        # Calculate EAR for both eyes
        left_eye_ear = eye_aspect_ratio(left_eye)
        right_eye_ear = eye_aspect_ratio(right_eye)

        # Calculate the average EAR
        eye_ear = (left_eye_ear + right_eye_ear) / 2.0

        # Calculate MAR for mouth
        mouth_mar = mouth_aspect_ratio(mouth)

        # Draw the landmarks and display EAR and MAR
        for n in range(36, 42):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)  # Left Eye landmarks in green

        for n in range(42, 48):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)  # Right Eye landmarks in green
        
        for n in range(48, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)  # Mouth landmarks in green
        
        # Display the average EAR and MAR on the frame
        cv2.putText(frame, f"Eye EAR: {eye_ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Mouth MAR: {mouth_mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the frame with landmarks
    cv2.imshow('Face, Eye EAR, and Mouth MAR Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()
